# 0. selenium 설치

In [1]:
!pip install chromedriver_autoinstaller

In [2]:
!pip install selenium

# 1. 라이브러리 불러오기

In [3]:
# 라이브러리
import numpy as np
import pandas as pd
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait


import chromedriver_autoinstaller

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm


# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time


# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 2. 팀 컬러 크롤링

##### 참고


- find_elements_by_tag_name:          Copy Element의 tag
- find_elements_by_xpath:             Copy XPath 또는 Copy full XPath
- find_elements_by_id:                Copy Element의 id attribute
- find_elements_by_class_name:        Copy Element의 class attribute
- find_elements_by_link_text:         Copy Element의 text
- find_elements_by_partial_link_text: Copy Element의 text중 일부
- find_elements_by_css_selector:      Copy selector


- '요소' 클릭:        	.click('')   
- '요소' 더블 클릭:	     .double_click('')   
- '텍스트' 입력:	         .send_keys('')  
- '요소'에 '텍스트' 입력:	  .send_keys_to_element('','')  
- '요소'로 마우스 이동:	  .move_to_element('')
- 스크롤 내리기:	         .execute_script("window.scrollto();")
- 실행:                  .perform()

> 팀 컬러 명, 적용단계, 적용 효과를 가져올 것 (+특성 팀 컬러의 경우는 추가적으로 '적용 선수' 목록까지 추가 )

### 2-1. 팀 컬러 목록 part

##### * 사용할 함수

In [4]:
# 크롬 윈도우 설정
def open_chrome(width ,length,start_time, end_time):
        ## 크롬 드라이버 위치 설정
        chrome_path = chromedriver_autoinstaller.install()


        ## 크롬 옵션
        options = webdriver.ChromeOptions()
        ## 크롬 윈도우 사이즈 조절
        options.add_argument(f"--window-size={width},{length}")

        driver = webdriver.Chrome(chrome_path, options=options)

        ## 실행할 시간 랜덤값 지정
        A = np.random.randint(start_time,end_time)
        
        return driver , A





# 팀 컬러 목록 위치 데이터 찾는 함수
def TC_list_crawling(x,driver):
        driver.find_element(By.XPATH, f'//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[{x}]').click()
        driver.find_element(By.CLASS_NAME, 'btn_search').click()
        driver.implicitly_wait(time_to_wait=20)

        elements = driver.find_elements(By.CLASS_NAME , 'name')

        return elements




## 팀 컬러 목록 크롤링 함수
def team_color_crawling_list(driver,A):
        # 팀 컬러 타입 리스트
        teamcolor_club = []
        teamcolor_nation = []
        teamcolor_reinforce = []
        teamcolor_relation = []
        teamcolor_special =[]
        
        # 홈페이지 열기
        url = 'https://fifaonline4.nexon.com/datacenter/teamcolor'
        driver.get(url)
        driver.implicitly_wait(time_to_wait=30)

        try:
                # 팝업창 닫기 (팝업창 오류 시 사용)
                driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/a/span').click()
                time.sleep(1)
        except:
                pass

        # 팀 컬러 정보 목록 저장
        for x in tqdm(range(2,7)):
                elements = TC_list_crawling(x,driver)
                time.sleep(A)

                ## 클럽
                if x == 2:
                        for element in elements:
                                teamcolor_club.append(element.text)
                
                elif x == 3:
                        for element in elements:
                                teamcolor_nation.append(element.text)
                
                elif x == 4:
                        for element in elements:
                                teamcolor_reinforce.append(element.text)
                                
                elif x == 5:
                        for element in elements:
                                teamcolor_relation.append(element.text)
                                
                elif x == 6:
                        for element in elements:
                                teamcolor_special.append(element.text)
                
                else:
                        print('not in range')
                        
                time.sleep(A)
                driver.find_element(By.CLASS_NAME, 'btn_reset').click()
        
        
        
        
        # 팀 컬러 목록 데이터 프레임 저장
        TC_club = pd.DataFrame(data = teamcolor_club, columns = ['team color'])
        TC_nation = pd.DataFrame(data = teamcolor_nation, columns = ['team color'])
        TC_reinforce = pd.DataFrame(data = teamcolor_reinforce, columns = ['team color'])
        TC_relation = pd.DataFrame(data = teamcolor_relation, columns = ['team color'])
        TC_special = pd.DataFrame(data = teamcolor_special, columns = ['team color'])
        
        
        # csv로 파일 저장
        TC_club.to_csv("./data/team_color_list/TC_club.csv", encoding='utf-8-sig', index = False)
        TC_nation.to_csv("./data/team_color_list/TC_nation.csv", encoding='utf-8-sig', index = False)
        TC_reinforce.to_csv("./data/team_color_list/TC_reinforce.csv", encoding='utf-8-sig', index = False)
        TC_relation.to_csv("./data/team_color_list/TC_relation.csv", encoding='utf-8-sig', index = False)
        TC_special.to_csv("./data/team_color_list/TC_special.csv", encoding='utf-8-sig', index = False)
    
    
        # 팀 컬러 타입 모음
        teamcolor_type = [teamcolor_club,teamcolor_nation,teamcolor_reinforce,teamcolor_relation,teamcolor_special]
        
        return teamcolor_type

##### 실행

In [ ]:
# 셀레니움 설정
## open_chrome(크롬창 가로 길이,크롬창 세로 길이, 랜덤 시간값 시작 범위, 랜덤 시간값 끝 범위) 
driver , A = open_chrome(1600,900,1.5,3.5) # driver: 웹드라이버 , A: 랜덤 시간 지정

# 각 팀 컬러 종류 별 리스트 정리
teamcolor_type = team_color_crawling_list(driver,A)

display(teamcolor_type)

### 2-2. 팀 컬러 세부 사항 part

##### 데이터 불러오기

In [13]:
# csv 파일 불러오기
TC_club = pd.read_csv('./data/team_color_list/TC_club.csv')
TC_nation = pd.read_csv('./data/team_color_list/TC_nation.csv')
TC_reinforce = pd.read_csv('./data/team_color_list/TC_reinforce.csv')
TC_relation = pd.read_csv('./data/team_color_list/TC_relation.csv')
TC_special = pd.read_csv('./data/team_color_list/TC_special.csv')

# 다시 리스트 형식으로 변경
teamcolor_club = TC_club['team color']
teamcolor_nation = TC_nation['team color']
teamcolor_reinforce = TC_reinforce['team color']
teamcolor_relation = TC_relation['team color']
teamcolor_special = TC_special['team color']

# 팀 컬러 타입 모음
teamcolor_type = [teamcolor_club,teamcolor_nation,teamcolor_reinforce,teamcolor_relation,teamcolor_special]

##### 사용할 코드

In [120]:
# css selector 위치 입력기
def location_inputor(driver,num,type):
    if type == 'div':
        temp_1 = driver.find_element(By.CSS_SELECTOR, 'div > div.lv_content > div.desc').text
        temp_2 = driver.find_element(By.CSS_SELECTOR, 'div > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lvu':
        temp_1 = driver.find_element(By.CSS_SELECTOR, f'div.level.lvu{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element(By.CSS_SELECTOR, f'div.level.lvu{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    elif type == 'div.level.lv':    
        temp_1 = driver.find_element(By.CSS_SELECTOR, f'div.level.lv{num} > div.lv_content > div.desc').text
        temp_2 = driver.find_element(By.CSS_SELECTOR, f'div.level.lv{num} > div.lv_content > div.ap_list').text
        temp_2 = temp_2.replace('\n',' / ').replace('-','')
        
    return temp_1, temp_2



# 팀 컬러 타입 별 적용 조건 & 효과 저장
def team_color_crawling_data(driver,A, teamcolor_type):
    # 변수 재 지정
    teamcolor_club = list(teamcolor_type[0])
    teamcolor_nation = list(teamcolor_type[1])
    teamcolor_reinforce = list(teamcolor_type[2])
    teamcolor_relation = list(teamcolor_type[3])
    teamcolor_special = list(teamcolor_type[4])
    
    
    # 홈페이지 열기
    url = 'https://fifaonline4.nexon.com/datacenter/teamcolor'
    driver.get(url)
    driver.implicitly_wait(time_to_wait=30)
    
    
    # 팝업창 닫기
    try:
        driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/a/span').click()
        time.sleep(A)
    except:
        pass

    
    count = 0

    for teamcolors in tqdm(teamcolor_type):
        # 크롤링한 결과를 저장할 리스트
        teamcolor_info = []
        step_1_require_player_nums = []
        step_2_require_player_nums = []
        step_3_require_player_nums = []
        step_4_require_player_nums = []
        step_1_effects = []
        step_2_effects = []
        step_3_effects = []
        step_4_effects = []
        
        for x in tqdm(teamcolors):
            try:
                # 팀 컬러 타입 선택
                if len(teamcolors) == len(teamcolor_club):
                    driver.find_element(By.XPATH, '//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[2]').click()
                elif len(teamcolors) == len(teamcolor_nation):
                    driver.find_element(By.XPATH, '//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[3]').click() 
                elif len(teamcolors) == len(teamcolor_reinforce):
                    driver.find_element(By.XPATH, '//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[4]').click()
                elif len(teamcolors) == len(teamcolor_relation):
                    driver.find_element(By.XPATH, '//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]').click() 
                elif len(teamcolors) == len(teamcolor_special):
                    driver.find_element(By.XPATH, '//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[6]').click()
                else: 
                    print('error: not match')
                
                
                # 팀 컬러 입력 & 검색
                driver.find_element(By.CLASS_NAME, 'ui-autocomplete-input').click()
                driver.find_element(By.CLASS_NAME, 'ui-autocomplete-input').send_keys(x)
                time.sleep(0.2)
                driver.find_element(By.CLASS_NAME, 'btn_search').click()
                driver.implicitly_wait(time_to_wait=10)
                
                
                # 단계 추출 (for 문 돌릴 갯수 정하기 용도)
                number = driver.find_element(By.CSS_SELECTOR, 'div.teamcolor_item_list > div > div.level').text
                number = int(number.replace('단계',''))
                time.sleep(A) 


                # 세부 설명 들어가기
                driver.find_element(By.CSS_SELECTOR, 'div:nth-child(1) > div.name').click()
                driver.find_element(By.CLASS_NAME, 'btn_detail_link').click()
                driver.implicitly_wait(time_to_wait=10)


                # 팀 컬러 설명 저장
                info = driver.find_element(By.CSS_SELECTOR, '#teamcolorPop > div > div.header > div > span').text
                teamcolor_info.append(info)
                time.sleep(0.5)
                
                
                
                # 단계 별 적용 조건 & 적용 효과 저장
                for num in range(1,5):
                    # 1단계
                    if num == 1:
                        if number == 1:
                            # 'num'과 아래의 타입 중 하나를 골라서 넣으세요!
                            # type = 'div', 'div.level.lvu', 'div.level.lv'
                            temp_1, temp_2 = __location_inputor(driver,num,'div')
                            
                            step_1_require_player_nums.append(temp_1)
                            step_1_effects.append(temp_2)
                            
                            
                        elif number > 1:   
                            if len(teamcolors) == len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lvu')
                            
                            elif len(teamcolors) != len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lv')
                        
                            step_1_require_player_nums.append(temp_1)
                            step_1_effects.append(temp_2)
                        else:
                            step_1_require_player_nums.append('-')
                            step_1_effects.append('-') 

                    # 2단계
                    elif num == 2:
                        if num <= number:
                            if len(teamcolors) == len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lvu')
                            
                            elif len(teamcolors) != len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lv')
                            
                            step_2_require_player_nums.append(temp_1)
                            step_2_effects.append(temp_2)
                            
                        else:
                            step_2_require_player_nums.append('-')
                            step_2_effects.append('-') 
                    
                    # 3단계
                    elif num == 3:
                        if num <= number:
                            if len(teamcolors) == len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lvu')
                            
                            elif len(teamcolors) != len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lv')
                            
                            step_3_require_player_nums.append(temp_1)
                            step_3_effects.append(temp_2)
                            
                        else:
                            step_3_require_player_nums.append('-')
                            step_3_effects.append('-') 

                    # 4단계
                    elif num == 4:
                        if num <= number:
                            if len(teamcolors) == len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lvu')
                            
                            elif len(teamcolors) != len(teamcolor_reinforce):
                                temp_1, temp_2 = __location_inputor(driver,num,'div.level.lv')
                            
                            step_4_require_player_nums.append(temp_1)
                            step_4_effects.append(temp_2)
                            
                        else:
                            step_4_require_player_nums.append('-')
                            step_4_effects.append('-') 

                    else:
                        print('error: location_inputor가 작동하지 않습니다.')
                        break
                    time.sleep(0.5)
                    
                # 세부 설명 나오기
                driver.find_element(By.XPATH, '//*[@id="teamcolorPop"]/div/div[1]/a').click()
                time.sleep(0.3)

                # 검색어 초기화 
                driver.find_element(By.CLASS_NAME,'btn_reset').click()
                time.sleep(A)
            
            except:
                print('error: 팀 컬러 선택 & 상세 정보 클릭')
                break
            
        # dict로 변형 & 데이터 프레임화
        dict_temp = { '팀 컬러': [val for val in teamcolors],
                    '팀 컬러 설명': [val for val in teamcolor_info],             
                    '1단계 적용 조건': [val for val in step_1_require_player_nums],
                    '2단계 적용 조건': [val for val in step_2_require_player_nums],
                    '3단계 적용 조건': [val for val in step_3_require_player_nums],
                    '4단계 적용 조건': [val for val in step_4_require_player_nums],
                    '1단계 효과': [val for val in step_1_effects], 
                    '2단계 효과': [val for val in step_2_effects],
                    '3단계 효과': [val for val in step_3_effects],
                    '4단계 효과': [val for val in step_4_effects]
                    }
        
        columns = ['팀 컬러','팀 컬러 설명','1단계 적용 조건','2단계 적용 조건','3단계 적용 조건','4단계 적용 조건','1단계 효과','2단계 효과','3단계 효과','4단계 효과']
        
        # # jupyter notebook ver.
        # teamcolor_naming = ['클럽_팀_컬러','국가_팀_컬러','강화_팀_컬러','관계_팀_컬러','스페셜_팀_컬러']
        # globals()[f'df_{teamcolor_naming[count]}'] = pd.DataFrame(data = dict_temp, columns=columns)
        # count += 1
        
        # py ver.
        if count == 0:
            df_teamcolor_club = pd.DataFrame(data = dict_temp, columns=columns)
        
        elif count == 1:
            df_teamcolor_nation = pd.DataFrame(data = dict_temp, columns=columns)
            
        elif count == 2:
            df_teamcolor_reinforce = pd.DataFrame(data = dict_temp, columns=columns)
            
        elif count == 3:
            df_teamcolor_relation = pd.DataFrame(data = dict_temp, columns=columns)
        
        elif count == 4:
            df_teamcolor_special = pd.DataFrame(data = dict_temp, columns=columns)
            
        else: 
            print('error: making dataframe')
        count += 1
    
    
    # csv파일로 저장
    df_teamcolor_club.to_csv('./data/team_color_crawling/클럽팀컬러.csv', encoding='utf-8-sig', index = False)
    df_teamcolor_nation.to_csv('./data/team_color_crawling/국가팀컬러.csv', encoding='utf-8-sig', index = False)
    df_teamcolor_reinforce.to_csv('./data/team_color_crawling/강화팀컬러.csv', encoding='utf-8-sig', index = False)
    df_teamcolor_relation.to_csv('./data/team_color_crawling/관계팀컬러.csv', encoding='utf-8-sig', index = False)
    df_teamcolor_special.to_csv('./data/team_color_crawling/스페셜팀컬러.csv', encoding='utf-8-sig', index = False)
    
    
    return df_teamcolor_club,df_teamcolor_nation,df_teamcolor_reinforce,df_teamcolor_relation,df_teamcolor_special

##### 시행

In [ ]:
driver, A = open_chrome(1600,900,1,3) 
df_teamcolor_club,df_teamcolor_nation,df_teamcolor_reinforce,df_teamcolor_relation,df_teamcolor_special = team_color_crawling_data(driver, A,teamcolor_type)

### 2-3. 관계 팀 컬러 - 선수 목록 part

In [ ]:
# overall 낮은 선수는 데이터가 없음!! 코드 수정 필요!!
# 오버롤 10씩 줄여 

In [9]:
# 정보 입력 코드
def insert_player_info(driver,path,num):
    driver.find_element_by_xpath(path).send_keys(Keys.RIGHT)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(Keys.RIGHT)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(Keys.RIGHT)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(Keys.BACKSPACE)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(Keys.BACKSPACE)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(Keys.BACKSPACE)
    time.sleep(0.03)
    driver.find_element_by_xpath(path).send_keys(str(num))
    time.sleep(0.03)

In [25]:
# chrome 창 열기
driver, A = open_chrome(1600,900,1,3) 


## 관계 팀 컬러 선수 시즌 & 이름 크롤링
# 팀컬러 기준을 만족하는 선수 리스트
TC_relation_plus = []

# 홈페이지 열기
url = 'https://fifaonline4.nexon.com/datacenter/teamcolor'
driver.get(url)


# 팝업창 닫기
try:
   driver.find_element(By.XPATH, '//*[@id="wrapper"]/div[1]/a/span').click()
   time.sleep(A)
except:
   pass


for x in tqdm(teamcolor_relation[:3]):
   each_teamcolor_players = []
   
   try:
      # 팀 컬러 타입 선택
      driver.find_element_by_xpath('//*[@id="sForm"]/div[3]/div[2]/div[2]/div/div/div[5]/label').click() 
      time.sleep(1)
      

      # 팀 컬러 입력 & 검색
      driver.find_element_by_class_name('ui-autocomplete-input').click()
      driver.find_element_by_class_name('ui-autocomplete-input').send_keys(x)
      time.sleep(0.2)
      driver.find_element_by_class_name('btn_search').click()
      driver.implicitly_wait(time_to_wait=10)


      # 세부 설명 들어가기
      driver.find_element_by_class_name('name').click()
      driver.find_element_by_class_name('btn_detail_link').click()
      driver.implicitly_wait(time_to_wait=10)
      
      
      # 오버롤 검색결과 최고 값 부터 10씩 감소 ~ 41 까지
      try:
         max_num = int(driver.find_element(By.XPATH, '//*[@id="ulPlayerList"]/li[1]/div[1]/div[3]/span').text)
      except:
         max_num = int(driver.find_element(By.XPATH, '//*[@id="ulPlayerList"]/li[1]/div[1]/div[3]/span[1]').text)
         
      min_num = max_num - 10
      
      while min_num > 40:
         
         try:
            # 팀 컬러 하나의 선수 목록을 저장할 리스트
            temp_TC_relation_plus =[]
            
            # 오버롤 초기화 및 입력
            # 최소값 입력
            insert_player_info(driver,'//*[@id="slider1"]/input[1]',min_num)
            # 최대값 입력
            insert_player_info(driver,'//*[@id="slider1"]/input[2]',max_num)
            
            driver.find_element(By.XPATH, '//*[@id="slider1"]/input[2]').send_keys(Keys.ENTER)
            
            
            
            # 조건 만족하는 선수 숫자 세기
            player_list = driver.find_elements_by_xpath('//*[@id="ulPlayerList"]')
            player_list_num = int(player_list[0].text.count('BP'))

            
            
            # 시즌 & 선수명 추출
            for num in range(1,player_list_num+1):         
               season_temp = driver.find_element_by_xpath(f'//*[@id="ulPlayerList"]/li[{num}]/div[1]/div[3]/div[1]/img')
               player_temp = driver.find_element_by_xpath(f'//*[@id="ulPlayerList"]/li[{num}]/div[1]/div[3]/div[2]')
               season = season_temp.get_attribute('src').split('/')[-1].replace('.png','')
               player = player_temp.text

               # 시즌 & 선수명 합치기
               total_temp = season+' '+player
               temp_TC_relation_plus.append(total_temp)
               
            each_teamcolor_players.extend(temp_TC_relation_plus)
          
         except:
            if driver.find_element(By.XPATH, '//*[@id="ulPlayerEmpty"]/li/div[2]').text == '검색된 선수가 없습니다.':
               break
         
         max_num -= 10
         min_num -= 10 
      
      each_teamcolor_players = list(set(each_teamcolor_players))
   
      TC_relation_plus.append(each_teamcolor_players)
      
     
      # 세부 설명 나오기
      driver.find_element_by_class_name('btn_close').click()
      time.sleep(1)

      # 검색어 초기화 
      driver.find_element_by_class_name('btn_reset').click()
      time.sleep(1)
   
   except:
      print('error: 팀 컬러 선택 & 상세 정보 클릭')
      break
      
             
# dict로 변형 & 데이터 프레임화
dict_temp = {  '팀 컬러': [val for val in teamcolor_relation[:3]],
               '시즌 & 이름': [val for val in TC_relation_plus]             }

columns = ['팀 컬러','시즌 & 이름']


df_TC_relation_plus = pd.DataFrame(data = dict_temp, columns=columns)
df_TC_relation_plus
   


  0%|          | 0/3 [00:00<?, ?it/s]

In [43]:
# 결과 확인
df_TC_relation_plus

,팀 컬러,시즌 & 이름
0,01-03 성남FC 리그3연패 베스트,"[TKL 김도훈, TKL 김상식, TKL 윤정환, TKL 김영철, TKL 이기형, ..."
1,02 수원삼성 더블,"[BWC 이운재, CAP 이운재, TKL 김두현, TKL 이운재, TKL 서정원, ..."
2,09 포항 스틸러스 더블,"[TKL 최효진, 20KLB 김광석, TKL 김기동, TKL 김광석, TKL 황지수..."
3,10 FC서울 더블,"[TKL 데얀, 21KLB 김태환, TKL 정조국, TKL 최효진, TKL 고요한,..."
4,13 포항 스틸러스 더블,"[21KLB 신진호, 20KLB 김광석, TKL 김광석, TKL 김승대, TKL 황..."
...,...,...
201,푸거 코리안 듀오,"[2012KH 구자철, 2012KH 지동원, TKL 지동원, LH 지동원, 2019..."
202,프랑스 1기 황금세대,"[ICON 지네딘 지단, BWC 지네딘 지단, RMCF 지네딘 지단, ICON 티에..."
203,프랑스 2기 황금세대,"[22TOTY 킬리안 음바페, 22TOTS 킬리안 음바페, BWC 앙투안 그리즈만,..."
204,프랑스 듀오,"[ICON 파트리크 비에이라, BWC 파트리크 비에이라, ICON 클로드 마켈렐레,..."


In [8]:
# csv 파일 불러오기
df_관계_팀_컬러 = pd.read_csv('./data/team_color_crawling/관계팀컬러.csv')

# # 관계 팀 컬러 적용 선수 목록 추가
# df_관계_팀_컬러_plus = pd.merge(df_관계_팀_컬러, df_TC_relation_plus, on = '팀 컬러', how = 'left')
# df_관계_팀_컬러_plus

# # csv 파일로 저장
# df_관계_팀_컬러_plus.to_csv('관계팀컬러.csv', encoding='utf-8-sig', index = False)